# Image Classification
In this project, we will be classifying images of birds into 200 classes using cnn model. Our goal is to use less number of parameters(max 10M) and improve accuracy.

In [1]:
import keras, os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import torch
import torchvision
import torch.nn as nn
from efficientnet_pytorch import EfficientNet


2024-03-06 03:16:20.593450: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-06 03:16:20.638120: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-06 03:16:20.638152: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-06 03:16:20.639349: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-06 03:16:20.646720: I tensorflow/core/platform/cpu_feature_guar

# Pre processing data

In [4]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.transforms.functional import InterpolationMode
from PIL import Image
import numpy as np
np.random.seed(0)

# Define the path to the directory containing your JPEG images
image_dir_test = "CUB_200_2011/test"
image_dir_train = "CUB_200_2011/train"

# Define the transformations to apply to the images
# You can adjust these transformations based on your specific requirements
transform = transforms.Compose([
    transforms.Resize(232, interpolation=transforms.InterpolationMode.BILINEAR),
    transforms.CenterCrop(224),
    transforms.ToTensor(),  # Convert PIL.Image to torch.Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
transform1 = transforms.Compose([
    transforms.Resize(232, interpolation=transforms.InterpolationMode.BILINEAR),
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.RandomRotation(10),      # Random rotation within -10 to 10 degrees
    transforms.CenterCrop(224),
    transforms.ToTensor(),  # Convert PIL.Image to torch.Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define a function to preprocess a single image
def preprocess_image(image_path, transform, transform1):
    image = Image.open(image_path).convert('RGB')  # Open the image using PIL
    # print(image.shape)
    image1 = transform(image)        # Apply the defined transformations
    image2 = transform1(image)
    return [image1, image2]

# Iterate through each image in the directory, preprocess it, and store it in a list
nameid = {}
idlabel = {}
with open('CUB_200_2011/images.txt', 'r') as file:
    for line in file:
        value, key = line.split()
        key = key.split('/')[1]
        nameid[key] = value

with open('CUB_200_2011/image_class_labels.txt', 'r') as file:
    for line in file:
        key, value = line.split()
        idlabel[key] = value

y_train = []

preprocessed_images = []
for filename in os.listdir(image_dir_train):
    if filename.endswith(".jpg"):
        image_path = os.path.join(image_dir_train, filename)
        preprocessed_image = preprocess_image(image_path, transform, transform1)
        preprocessed_images.append(preprocessed_image[0])
        preprocessed_images.append(preprocessed_image[1])
        y_train.append(idlabel[nameid[filename]])
        y_train.append(idlabel[nameid[filename]])

# Concatenate the preprocessed images along the batch dimension to create a single tensor
preprocessed_images_tensor = torch.stack(preprocessed_images)

# Save the preprocessed images tensor to a file
torch.save(preprocessed_images_tensor, "preprocessed_images_train_aug.pt")

y_test = []
preprocessed_images = []
for filename in os.listdir(image_dir_test):
    if filename.endswith(".jpg"):
        image_path = os.path.join(image_dir_test, filename)
        preprocessed_image = preprocess_image(image_path, transform, transform1)
        preprocessed_images.append(preprocessed_image[0])
        y_test.append(idlabel[nameid[filename]])

# Concatenate the preprocessed images along the batch dimension to create a single tensor
preprocessed_images_tensor = torch.stack(preprocessed_images)

# Save the preprocessed images tensor to a file
torch.save(preprocessed_images_tensor, "preprocessed_images_test_aug.pt")

import pickle
with open("test_label_aug.pkl", "wb") as file:
    pickle.dump(y_test, file)
with open("train_label_aug.pkl", "wb") as file:
    pickle.dump(y_train, file)


In [1]:
import pickle
with open("test_label_aug.pkl", "wb") as file:
    pickle.dump(y_test, file)
with open("train_label_aug.pkl", "wb") as file:
    pickle.dump(y_train, file)
        

NameError: name 'y_test' is not defined

# Defining model

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from efficientnet_pytorch import EfficientNet
import pickle
from tqdm import tqdm

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pretrained EfficientNet B2 model
model = EfficientNet.from_pretrained('efficientnet-b2')
model.to(device)

# Define your custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data_file, labels):
        self.data = torch.load(data_file)  # Load data from .pt file
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Get data and label for the given index
        data_item = self.data[idx].to(device)
        label = self.labels[idx].to(device)
        return data_item, label

# Load the pickle file into a list
with open("list.pkl", "rb") as file:
    y_train_old = pickle.load(file)

y_train = torch.tensor(y_train_old, dtype=torch.long).to(device)

dataset = CustomDataset('preprocessed_images_train.pt', y_train)

batch_size = 32
shuffle = True  # You can set this to True for training
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Train the model or load your own data for fine-tuning
# Example:
training_loss = []
print("Training...")
for inputs, labels in tqdm(dataloader, desc="Training"):
    inputs = inputs.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    with torch.no_grad():
        outputs = model(inputs)
    loss = criterion(outputs, labels)
    training_loss.append(loss.item())
    loss.backward()
    optimizer.step()

print("Training completed.")


Using device: cuda
Loaded pretrained weights for efficientnet-b2
